### Basic training and testing

In [14]:
using MLJ
using DataFrames

task = load_boston()
X, y = task()

train, test = partition(eachindex(y), 0.7); # 70:30 split

A *model* is a container for hyperparameters:

In [15]:
knn_model=KNNRegressor(K=10)

KNNRegressor(K = 10,
             metric = MLJ.KNN.euclidean,
             kernel = MLJ.KNN.reciprocal,) @ 1…81

Wrapping the model in data creates a *machine* which will store training outcomes (called *fit-results*):

In [16]:
knn = machine(knn_model, X, y)

Machine @ 1…04


Training on the training rows and evaluating on the test rows:

In [17]:
fit!(knn, rows=train)
yhat = predict(knn, X[test,:])
rms(y[test], yhat)

┌ Info: Training Machine @ 1…04.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:110


8.090639098853249

Or, in one line:

In [18]:
evaluate!(knn, resampling=Holdout(fraction_train=0.7))

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/resampling.jl:91


8.090639098853249

Changing a hyperparameter and re-evaluating:

In [19]:
knn_model.K = 20
evaluate!(knn, resampling=Holdout(fraction_train=0.7))

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/resampling.jl:91


8.41003854724935

### Systematic tuning as a model wrapper

A simple example of a composite model is a homogeneous ensemble. Here's a bagged ensemble model for 20 K-nearest neighbour regressors:

In [20]:
ensemble_model = EnsembleModel(atom=knn_model, n=20) 

MLJ.DeterministicEnsembleModel(atom = KNNRegressor @ 1…81,
                               weights = Float64[],
                               bagging_fraction = 0.8,
                               rng = MersenneTwister(UInt32[0x900399a4, 0xfa44b3f7, 0xd25ce0d5, 0x94384da6]),
                               n = 20,
                               parallel = true,
                               out_of_bag_measure = Any[],) @ 1…14

Let's simultaneously tune the ensemble's `bagging_fraction` and the K-nearest neighbour hyperparameter `K`. Since one of our models is a field of the other, we have nested hyperparameters:

In [21]:
params(ensemble_model) # a named tuple (nested)

(atom = (K = 20,
         metric = MLJ.KNN.euclidean,
         kernel = MLJ.KNN.reciprocal,),
 weights = Float64[],
 bagging_fraction = 0.8,
 rng = MersenneTwister(UInt32[0x900399a4, 0xfa44b3f7, 0xd25ce0d5, 0x94384da6]),
 n = 20,
 parallel = true,
 out_of_bag_measure = Any[],)

To define a tuning grid, we construct ranges for the two parameters and collate these ranges following the same pattern above (omitting parameters that don't change):

In [22]:
B_range = range(ensemble_model, :bagging_fraction, lower= 0.5, upper=1.0, scale = :linear)
K_range = range(knn_model, :K, lower=1, upper=100, scale=:log10)
nested_ranges = (atom = (K = K_range,), bagging_fraction = B_range)

(atom = (K = NumericRange{K} @ 1…78,),
 bagging_fraction = NumericRange{bagging_fraction} @ 6…33,)

Now we choose a tuning strategy, and a resampling strategy (for estimating performance), and wrap these strategies around our ensemble model to obtain a new model:

In [23]:
tuning = Grid(resolution=8)
resampling = CV(nfolds=6)

tuned_ensemble_model = TunedModel(model=ensemble_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges)

MLJ.DeterministicTunedModel(model = DeterministicEnsembleModel @ 1…14,
                            tuning = Grid @ 6…89,
                            resampling = CV @ 3…64,
                            measure = nothing,
                            operation = StatsBase.predict,
                            nested_ranges = (atom = (K = NumericRange{K} @ 1…78,), bagging_fraction = NumericRange{bagging_fraction} @ 6…33),
                            minimize = true,
                            full_report = true,) @ 5…71

Fitting the corresponding machine tunes the underlying model (in this case an ensemble) and retrains on all supplied data:

In [24]:
tuned_ensemble = machine(tuned_ensemble_model, X[train,:], y[train])
fit!(tuned_ensemble);

┌ Warning: No measure specified. Using measure=MLJ.rms. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:99
┌ Info: Training Machine @ 1…83.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/machines.jl:110
Iterating over a 64-point grid: 100%[=========================] Time: 0:00:22
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/MLJ/src/tuning.jl:179


In [25]:
fp = fitted_params(tuned_ensemble)

(best_model = DeterministicEnsembleModel @ 7…60,)

In [26]:
@show fp.best_model.bagging_fraction
@show fp.best_model.atom.K;

(fp.best_model).bagging_fraction = 0.7142857142857143
((fp.best_model).atom).K = 52
